In [4]:
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# array for drawing circle, polar coordinates
theta = np.linspace( 0 , 2 * np.pi , 150 )

# wavelength range
wavelength = np.linspace(1500e-9, 1600e-9, 1000)

# coupling + attenuation coefficients
r1 = .85
r2 = .85
a = 1

# transmission functions
def transmission_add(r1, r2, a, p):
    return (
        ((r2**2*a**2)-(2*r1*r2*a*np.cos(p))+(r1**2))/(1-(2*r1*r2*a*np.cos(p))+((r1*r2*a)**2))
    )

def transmission_drop(r1, r2, a, p):
    return (
    ((1-(r1**2))*(1-(r2**2))*a)/(1-(2*r1*r2*a*np.cos(p))+(r1*r2*a)**2)
    )

# vectorize functions to operate over arrays
transmission_add = np.vectorize(transmission_add)
transmission_drop = np.vectorize(transmission_drop)

# adjust radius of ring - path length
def ring_size(rad):
    a = rad * np.cos( theta )
    b = rad * np.sin( theta )
    return a, b


app = Dash(__name__)

app.layout = html.Div([
    html.Div(
        children=[
        dcc.Graph(id='graph-with-slider'),
        ],
        style={"width": "49%", "display": "inline-block"}
    ),
    html.Div(
        children=[
        dcc.Graph(id='graph2-with-slider'),
        ],
        style={"width": "49%", "display": "inline-block"}
    ),
    
    #radius input - will affect path length and therefore phase shift
    dcc.Slider(
        4,
        14,
        step=.1,
        value=5,
        marks=None,
        tooltip={"placement": "bottom", "always_visible": True},
        id='slider'
    )
])


@callback(
    Output('graph-with-slider', 'figure'),
    Output('graph2-with-slider', 'figure'),
    Input('slider', 'value'))
def update_figure(value):
    
    data = ring_size(value)
    
    # path length -> circumference -> 2 * pi * radius (um)
    path_length = 2*np.pi*(value)*1e-6
    # propagation constant -> 2*pi*neff/wavelength NOTE: neff is currently not included, need to tweak
    k = (2*np.pi)/wavelength
    # phase shift = k * L
    phase_shift = k * path_length
    
    # compute points for transmission at 1550nm
    k_1550nm = (2*np.pi)/1550e-9
    ps_1550nm = k_1550nm * path_length
    
    fig = make_subplots(rows=1, cols=2)
    
    fig = go.Figure(
    data=[
        go.Scatter(x=data[0], y=data[1]),
        go.Scatter(x=[-value, value], y=[0, 0]),
        go.Scatter(x=[-80, 80], y=[value, value]),
        go.Scatter(x=[-80, 80], y=[-value, -value])
    ],
    layout=go.Layout(
        xaxis=dict(range=[-25, 25], autorange=False, showticklabels=False),
        yaxis=dict(range=[-25, 25], autorange=False, showticklabels=False),
        title="Ring Radius"
    )
    )
    
    fig.add_trace(go.Scatter(
        x=[0, 0],
        y=[0, 0],
        mode="text",
        name="Text",
        text=[f'{value} um'],
        textposition="top center",
        textfont=dict(family="Arial Black")
        )
    )
    
    fig.add_annotation(
        x=10,
        y=value,
        xref="x",
        yref="y",
        text=f'Transmission (Through Port): \n {round(transmission_add(r1, r2, a, ps_1550nm)*100, 2)}%',
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=0,
        ay=-60,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#40456b",
        opacity=0.8
        )
    
    fig.add_annotation(
        x=-10,
        y=-value,
        xref="x",
        yref="y",
        text=f'Transmission (Drop Port): \n {round(transmission_drop(r1, r2, a, ps_1550nm)*100, 2)}%',
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=0,
        ay=60,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#40456b",
        opacity=0.8
        )

    fig.update_layout(transition_duration=100,
                     yaxis_scaleanchor="x",
                     template="plotly_white",
                     showlegend=False)
    fig.update_traces(marker=dict(color='#906fd9'))
    fig2 = go.Figure(
    data=[
        go.Scatter(x=wavelength, y=transmission_drop(r1, r2, a, phase_shift)),
        go.Scatter(x=[1550e-9], y=[transmission_drop(r1, r2, a, ps_1550nm)]),
        go.Line(
            x=[1500e-9, 1550e-9], 
            y=[transmission_drop(r1, r2, a, ps_1550nm),
               transmission_drop(r1, r2, a, ps_1550nm)],
            mode='lines',
            line = dict(shape = 'linear', color = 'black', width= 2, dash = 'dash')
        ),
        go.Line(
            x=[1550e-9, 1550e-9], 
            y=[0,
               transmission_drop(r1, r2, a, ps_1550nm)],
            mode='lines',
            line = dict(shape = 'linear', color = 'black', width= 2, dash = 'dash')
        )
    ])
    
    fig2.update_layout(
        transition_duration=100,
        template="plotly_white",
        title="Transmission",
        xaxis_title="Wavelength (um)",
        yaxis_title="Transmission (%)",
        showlegend=False
    )

    return fig, fig2


if __name__ == '__main__':
    app.run(debug=True)